## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import weather_api_key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Escondido,MX,15.8500,-97.0667,77.83,69,20,0.00,few clouds
1,1,Vaini,TO,-21.2000,-175.2000,76.15,79,100,12.59,overcast clouds
2,2,Aljezur,PT,37.3191,-8.8033,74.41,57,6,14.47,clear sky
3,3,Taicheng,CN,38.7310,113.2280,64.11,20,0,8.14,clear sky
4,4,Bredasdorp,ZA,-34.5322,20.0403,69.26,51,39,11.83,scattered clouds


# 2. Prompt the user to enter minimum and maximum temperature criteria 
Max= input ("enter maximum temperture :")
Min= input ("enter minimum temperture :")

In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Escondido,MX,15.8500,-97.0667,77.83,69,20,0.00,few clouds
1,1,Vaini,TO,-21.2000,-175.2000,76.15,79,100,12.59,overcast clouds
6,6,Lazaro Cardenas,MX,17.9583,-102.2000,76.05,84,62,3.56,broken clouds
10,10,Sestri Levante,IT,44.2732,9.3968,78.06,82,46,8.01,scattered clouds
15,15,Cape Town,ZA,-33.9258,18.4232,75.11,46,43,3.00,scattered clouds
18,18,Victoria,HK,22.2855,114.1577,87.19,84,100,11.01,overcast clouds
31,31,Lorengau,PG,-2.0226,147.2712,80.49,72,6,5.19,clear sky
39,39,Hilo,US,19.7297,-155.0900,75.27,97,100,3.44,light rain
40,40,Cairns,AU,-16.9167,145.7667,75.07,93,79,8.05,broken clouds
41,41,Marsh Harbour,BS,26.5412,-77.0636,79.30,78,39,19.93,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
city_data_df.head()
np.where(pd.isnull(city_data_df))

(array([163, 283, 351, 365, 408], dtype=int64),
 array([2, 2, 2, 2, 2], dtype=int64))

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_df.dropna(inplace=True)
city_data_df

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Escondido,MX,15.8500,-97.0667,77.83,69,20,0.00,few clouds
1,1,Vaini,TO,-21.2000,-175.2000,76.15,79,100,12.59,overcast clouds
2,2,Aljezur,PT,37.3191,-8.8033,74.41,57,6,14.47,clear sky
3,3,Taicheng,CN,38.7310,113.2280,64.11,20,0,8.14,clear sky
4,4,Bredasdorp,ZA,-34.5322,20.0403,69.26,51,39,11.83,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
694,694,Kaoma,ZM,-14.7833,24.8000,74.68,29,5,11.90,clear sky
695,695,Gien,FR,47.6933,2.6309,71.44,47,39,17.65,scattered clouds
696,696,Erzincan,TR,39.7500,39.5000,87.10,16,0,6.91,clear sky
697,697,Evensk,RU,61.9500,159.2333,41.63,94,100,1.95,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Escondido,MX,77.83,few clouds,15.8500,-97.0667,
1,Vaini,TO,76.15,overcast clouds,-21.2000,-175.2000,
2,Aljezur,PT,74.41,clear sky,37.3191,-8.8033,
3,Taicheng,CN,64.11,clear sky,38.7310,113.2280,
4,Bredasdorp,ZA,69.26,scattered clouds,-34.5322,20.0403,
5,Khatanga,RU,30.25,overcast clouds,71.9667,102.5000,
6,Lazaro Cardenas,MX,76.05,broken clouds,17.9583,-102.2000,
7,Kem,RU,62.56,scattered clouds,64.9555,34.5793,
8,Shanghai,CN,73.26,broken clouds,31.2222,121.4581,
9,Barrow,US,24.82,overcast clouds,71.2906,-156.7887,


In [8]:
# Iterate through the DataFrame.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Escondido,MX,77.83,few clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
1,Vaini,TO,76.15,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
2,Aljezur,PT,74.41,clear sky,37.3191,-8.8033,
3,Taicheng,CN,64.11,clear sky,38.7310,113.2280,五台山康旅酒店
4,Bredasdorp,ZA,69.26,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor


In [10]:
hotel_df.head()
np.where(pd.isnull(hotel_df))

(array([], dtype=int64), array([], dtype=int64))

In [11]:
hotel_df.isna().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [12]:
nan_value = float("NaN")


hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(subset = ["Hotel Name"], inplace=True)


In [13]:
# 7. Drop the rows where there is no Hotel Name.


hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Escondido,MX,77.83,few clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
1,Vaini,TO,76.15,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
3,Taicheng,CN,64.11,clear sky,38.7310,113.2280,五台山康旅酒店
4,Bredasdorp,ZA,69.26,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor
9,Barrow,US,24.82,overcast clouds,71.2906,-156.7887,King Eider Inn
12,New Norfolk,AU,47.70,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages
15,Cape Town,ZA,75.11,scattered clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
16,Yuanping,CN,69.40,clear sky,38.7153,112.7575,Yuanping Xinghe Guesthouse
575,Bulaevo,KZ,59.36,broken clouds,54.9056,70.4439,"Игровой клуб ""ДОМ ИГР Z"""
648,Ambilobe,MG,82.18,clear sky,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe


In [14]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
 lat = row["Lat"]
 lng = row["Lng"]
 params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
      
 base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   # Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        # Create an empty list to hold the weather data.

    # Set the parameters to search for a hotel in Paris.

# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotels
# Search for hotels and add them in the weather dataframe.

#     print(hotels)
try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
